In [1]:
import requests
import json
import sys
from pyspark.sql import SparkSession

# --- Inicialização ---
spark = SparkSession.builder.appName("ETL_01_Bronze").getOrCreate()

# --- Extração (Extract) - CÓDIGO ORIGINAL ---
both = "SEX_BTSX"
female = "SEX_FMLE"
male = "SEX_MLE"

# URLs Originais
URL_SUICIDIO_B = f"https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq '{both}' and Dim2 eq 'AGEGROUP_YEARSALL'"
URL_SUICIDIO_F = f"https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq '{female}' and Dim2 eq 'AGEGROUP_YEARSALL'"
URL_SUICIDIO_M = f"https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq '{male}' and Dim2 eq 'AGEGROUP_YEARSALL'"

URL_DEPRESSAO_B = f"https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq '{both}'"
URL_DEPRESSAO_F = f"https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq '{female}'"
URL_DEPRESSAO_M = f"https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim1 eq '{male}'"

headers = { "Content-Type": "application/json" }

# Função Original
def extrair_api(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        dados = response.json().get("value",[])
        print(f"Dados coletados com sucesso! Total: {len(dados)}")
        return dados
    except Exception as e:
        print(f"Erro ao chamar a API: {e}")
        return []

# Execução da Extração
print("Extraindo dados de cada api:")
# Dicionário auxiliar para automatizar o salvamento
datasets = {
    "suicidio_b": extrair_api(URL_SUICIDIO_B),
    "suicidio_f": extrair_api(URL_SUICIDIO_F),
    "suicidio_m": extrair_api(URL_SUICIDIO_M),
    "depressao_b": extrair_api(URL_DEPRESSAO_B),
    "depressao_f": extrair_api(URL_DEPRESSAO_F),
    "depressao_m": extrair_api(URL_DEPRESSAO_M)
}

# --- Salvar Bronze (JSON Bruto) ---
# Necessário para persistir o dado para o próximo script
for nome, dados in datasets.items():
    if dados:
        # Transforma a lista de dicts em JSON string e salva
        rdd = spark.sparkContext.parallelize([json.dumps(d) for d in dados])
        df_raw = spark.read.json(rdd)
        
        caminho_bronze = f"hdfs://namenode:9000/datalake/bronze/{nome}"
        df_raw.write.mode("overwrite").json(caminho_bronze)
        print(f"Salvo em Bronze: {caminho_bronze}")

spark.stop()

Extraindo dados de cada api:
Dados coletados com sucesso! Total: 185
Dados coletados com sucesso! Total: 185
Dados coletados com sucesso! Total: 185
Dados coletados com sucesso! Total: 185
Dados coletados com sucesso! Total: 185
Dados coletados com sucesso! Total: 185
Salvo em Bronze: hdfs://namenode:9000/datalake/bronze/suicidio_b
Salvo em Bronze: hdfs://namenode:9000/datalake/bronze/suicidio_f
Salvo em Bronze: hdfs://namenode:9000/datalake/bronze/suicidio_m
Salvo em Bronze: hdfs://namenode:9000/datalake/bronze/depressao_b
Salvo em Bronze: hdfs://namenode:9000/datalake/bronze/depressao_f
Salvo em Bronze: hdfs://namenode:9000/datalake/bronze/depressao_m
